This notebook is for any new inputs to predict probabilities of genres and decades - this was originally used as a version to test before it was implemented in a .py file for the website

### Load Packages and Models

In [1]:
#load in packages

import pandas as pd
import graphlab as gl
import pickle

In [2]:
#load in trained model from Final_Model_Build

with open('./technical_analysis/svm_ddecade_model.pkl', 'r') as picklefile:
    svm_d_model = pickle.load(picklefile)

with open('./technical_analysis/svm_g_model.pkl', 'r') as picklefile:
    svm_g_model = pickle.load(picklefile)

### Do transformations on an input

In [3]:
# read in a sample input
test_pic = gl.Image("./sample_images/eddie.jpg")

#need a dataframe so second image is a dummy image
test_pic = [gl.image_analysis.resize(test_pic, 100, 100, 3, decode=True), gl.Image("./image_data_dump/1.jpg")]
test_pic = gl.SFrame(test_pic)

This non-commercial license of GraphLab Create for academic use is assigned to katty.lein@gmail.com and will expire on August 13, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1472583913.log


In [4]:
#begin to load in deep feature extractor
extractor = gl.feature_engineering.DeepFeatureExtractor(features = "X1", model='auto')

Downloading https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45/dir_archive.ini to /var/tmp/graphlab-katty/3639/51d43c23-9bcd-4520-a011-df58a47dd363.ini

Downloading https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45/objects.bin to /var/tmp/graphlab-katty/3639/cd05b9da-1831-4a09-bd51-b5c48a7dc105.bin

In [5]:
#fit to new input
extractor_g = extractor.fit(test_pic)
features_g = extractor_g.transform(test_pic)

Images being resized.

### Predict Probabilities and Transform to Dictionary for JSON

In [11]:
#run predict for decade for new input
d_values = svm_d_model.predict_proba(features_g[0]["deep_features.X1"])
d_keys = svm_d_model.classes_

/Users/katty/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [12]:
#transform outputs of the probabilities for decades into a dictionary to be read into JSON for application
d_values = d_values.tolist()
d_values = [round(item,3) for sublist in d_values for item in sublist]
d_dict = dict(zip(d_keys, d_values))

In [13]:
#run predict for genre for new input
g_values = svm_g_model.predict_proba(features_g[0]["deep_features.X1"])
g_keys = svm_g_model.classes_

/Users/katty/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/katty/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/katty/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1,

In [14]:
#transform outputs of the probabilities for genres into a dictionary to be read into JSON for application
g_values = g_values.tolist()
g_values = [round(item,3) for sublist in g_values for item in sublist]
g_dict = dict(zip(g_keys, g_values))

In [15]:
result = {'decade': d_dict, 'genre': g_dict}

### Final Output for Website

In [16]:
result

{'decade': {'1980 to 2000': 0.803, '2000 to Present': 0.197},
 'genre': {'Alternative': 0.066,
  'Blues': 0.078,
  'Country': 0.04,
  'Dance': 0.092,
  'Electronic': 0.078,
  'HipHop': 0.057,
  'Jazz': 0.094,
  'Pop': 0.061,
  'RB Soul': 0.225,
  'Reggae': 0.05,
  'Rock': 0.047,
  'Singer/Songwriter': 0.065,
  'World': 0.046}}